In [226]:
import mitsuba as mi
import os
mi.set_variant("scalar_rgb")

# Create an alias for convenience
from mitsuba import ScalarTransform4f as T

def load_sensor(r, phi, theta, target):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = T.rotate([0, 0, 1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 25,
        # -1 0 0 0 0 1 0 1 0 0 -1 6.8 0 0 0 1
        'to_world': T.look_at(
            origin=origin,
            target=target,
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 10
        },
        'film': {
            'type': 'hdrfilm',
            'width': 64,
            'height': 64,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgb',
        },
    })

sensor_count = 6

radius = 5
phis = [ 140 - (i*20) for i in range(sensor_count)]
theta = 22

sensors = [load_sensor(radius, phi, theta, [0, 1, 0]) for phi in phis]

In [227]:
def prepare_data(scene_file, max_depth, data_spp, ref_spp, sensors, output_folder):
        
    os.makedirs(output_folder, exist_ok=True)
    
    scene = mi.load_file(scene_file)

    ref_integrator = mi.load_dict({'type': 'path', 'max_depth': max_depth})
    gnn_integrator = mi.load_dict({'type': 'pathgnn', 'max_depth': max_depth})
        
    # generate gnn file data and references
    ref_images = []
    output_gnn_files = []
    
    print(f'Generation of {len(sensors)} views for `{scene_file}`')
    for view_i, sensor in enumerate(sensors):
        
        print(f'Generating data for view n°{view_i+1}')
        ref_images.append(mi.render(scene, spp=ref_spp, integrator=ref_integrator, sensor=sensor))
        
        print(f' -- reference of view n°{view_i+1} generated...')
        params = mi.traverse(scene)
        gnn_log_filename = f'{output_folder}/gnn_file_{view_i}.path'
        params['logfile'] = gnn_log_filename
        params.update();
        
        if not os.path.exists(gnn_log_filename):
            mi.render(scene, spp=data_spp, integrator=gnn_integrator, sensor=sensor)
        print(f' -- GNN data of view n°{view_i+1} generated...')
        
        output_gnn_files.append(gnn_log_filename)
        
    return output_gnn_files, ref_images

In [228]:
scene_file = 'scenes/cornell-box/scene.xml'
gnn_files, ref_images = prepare_data(scene_file, 
                                max_depth = 5, 
                                data_spp = 10, 
                                ref_spp = 1000, 
                                sensors = sensors, 
                                output_folder = 'data/model1')

Generation of 6 views for `scenes/cornell-box/scene.xml`
Generating data for view n°1
 -- reference of view n°1 generated...
 -- GNN data of view n°1 generated...
Generating data for view n°2
 -- reference of view n°2 generated...
 -- GNN data of view n°2 generated...
Generating data for view n°3
 -- reference of view n°3 generated...
 -- GNN data of view n°3 generated...
Generating data for view n°4
 -- reference of view n°4 generated...
 -- GNN data of view n°4 generated...
Generating data for view n°5
 -- reference of view n°5 generated...
 -- GNN data of view n°5 generated...
Generating data for view n°6
 -- reference of view n°6 generated...
 -- GNN data of view n°6 generated...


In [229]:
from mignn.container import SimpleLightGraphContainer

containers = []
for gnn_i, gnn_file in enumerate(gnn_files):
    ref_image = ref_images[gnn_i]
    container = SimpleLightGraphContainer.fromfile(gnn_file, scene_file, ref_image, verbose=True)
    containers.append(container)

Load of `data/model1/gnn_file_0.path` in progress: 100.00%
Load of `data/model1/gnn_file_1.path` in progress: 100.00%
Load of `data/model1/gnn_file_2.path` in progress: 100.00%
Load of `data/model1/gnn_file_3.path` in progress: 100.00%
Load of `data/model1/gnn_file_4.path` in progress: 100.00%
Load of `data/model1/gnn_file_5.path` in progress: 100.00%


In [230]:
from mignn.manager import LightGraphManager

# build connections individually
for container in containers:
    container.build_connections(n_graphs=10, n_nodes_per_graphs=5, n_neighbors=5, verbose=True)
    
merged_graph_container = LightGraphManager.fusion(containers)

/home/jerome/Documents/Research/Development/Rendering/mitsuba3/build-gnn/python/drjit/router.py:1753: RuntimeWarning: divide by zero encountered in divide
  return 1.0 / arg
/home/jerome/Documents/Research/Development/Rendering/mitsuba3/build-gnn/python/drjit/generic.py:194: RuntimeWarning: invalid value encountered in scalar multiply
  ar[i] = a0[i] * a1


KeyboardInterrupt: 

In [ ]:
print(merged_graph_container)

In [ ]:
def embedded_data(x_node, L=6):
    """
    Gets a base embedding for one dimension with sin and cos intertwined
    """
    n_elements = x_node.shape[0]
    powers = torch.pow(2., torch.arange(L))
    
    emb_data = []
    
    for p in x_node:
        coord_data = torch.empty(0)
        for coord in p:
            x_cos = torch.cos(coord * powers)
            x_sin = torch.cos(coord * powers)
            coord_emb = torch.cat((coord.unsqueeze(0), x_cos, x_sin), 0)
            coord_data = torch.cat((coord_data, coord_emb), 0)
        emb_data.append(coord_data)
    
    return torch.stack(emb_data)

In [ ]:
from torch_geometric.data import Data

data_list = []

for key in merged_graph_container.keys():
    graphs = merged_graph_container.graphs_at(key)
    for graph in graphs:
        torch_data = graph.data.to_torch()
        
        # do embedding
        data = Data(x = embedded_data(torch_data.x), 
                edge_index = torch_data.edge_index,
                y = torch_data.y,
                edge_attr = torch_data.edge_attr,
                pos = torch_data.pos)
        data_list.append(data)

In [ ]:
print(data_list[0].x)
print(data_list[0].edge_attr)

In [2]:
from torch_geometric.data import InMemoryDataset
import torch

class PathLightDataset(InMemoryDataset):
    def __init__(self, root, data_list=None, transform=None):
        self.data_list = data_list
        super().__init__(root, transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['data.pt']
    
    def download(self):
        pass

    def process(self):
        torch.save(self.collate(self.data_list), self.processed_paths[0])

In [58]:
pl_dataset = PathLightDataset('data/datasets/cornell', data_list)

NameError: name 'data_list' is not defined

In [211]:
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

# transform applied only when loaded
dataset = PathLightDataset(root='data/datasets/cornell')

split_index = int(len(dataset) * 0.8)
train_dataset = dataset[:split_index]
test_dataset = dataset[split_index:]

In [61]:
# normalize data
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler(feature_range=(-1, 1)).fit(train_dataset.data.x)
y_scaler = MinMaxScaler(feature_range=(-1, 1)).fit(train_dataset.data.y)

#x_mean = train_dataset.data.x.mean(dim=0, keepdim=True)
#x_std = train_dataset.data.x.std(dim=0, keepdim=True)
#print(x_mean, x_std)

#y_mean = train_dataset.data.y.mean(dim=0, keepdim=True)
#y_std = train_dataset.data.y.std(dim=0, keepdim=True)
#print(y_mean, y_std)

In [62]:
print(train_dataset[1].y)
print(y_scaler.transform(train_dataset[1].y))

tensor([[0.0627, 0.0411, 0.0111]])
[[-0.99259065 -0.99311359 -0.99443767]]


In [63]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [64]:
print(train_dataset[0].pos)
print(train_dataset[0].edge_attr)

tensor([[-1.4313,  1.2040,  4.6265],
        [ 0.6721,  1.2583, -1.0000],
        [ 0.0000,  0.0000,  0.0000]])
tensor([6.0071,    inf])


In [208]:
from torch.nn import Linear, Conv2d
import torch.nn.functional as F
from torch_geometric.nn import GraphConv 
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, L=6):
        
        super(GCN, self).__init__()
        self._n_features = dataset.num_node_features
        self.conv1 = GraphConv(self._n_features * (L * 2) + self._n_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels * 4)
        self.lin1 = Linear(hidden_channels * 4, hidden_channels)
        self.lin2 = Linear(hidden_channels, 3)

    def forward(self, x, edge_attr, edge_index, batch):
        # 1. Obtain node embeddings 
        x_node = self.conv1(x, edge_index)
        x_node = x_node.relu()
        x_node = self.conv2(x_node, edge_index)
        x_node = x_node.relu()
        x_node = self.conv3(x_node, edge_index)

        # 2. Readout layer
        h_node = global_mean_pool(x_node, batch)  # [batch_size, hidden_channels]
        
        # 3. Apply a final classifier
        x = F.dropout(h_node, p=0.5, training=self.training)
        x = self.lin1(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        
        return x

model = GCN(hidden_channels=256)
print(model)
print(f'Number of params: {sum(p.numel() for p in model.parameters())}')

GCN(
  (conv1): GraphConv(78, 256)
  (conv2): GraphConv(256, 256)
  (conv3): GraphConv(256, 1024)
  (lin1): Linear(in_features=1024, out_features=256, bias=True)
  (lin2): Linear(in_features=256, out_features=3, bias=True)
)
Number of params: 960003


In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.HuberLoss()
from torchmetrics import R2Score
r2 = R2Score()

def train(epoch_id):
    model.train()

    error = 0
    r2_error = 0
    for b_i, data in enumerate(train_loader):  # Iterate in batches over the training dataset.
        
        # normalize data
        x_data = embedded_data(torch.tensor(x_scaler.transform(data.x), dtype=torch.float))
        y_data = torch.tensor(y_scaler.transform(data.y), dtype=torch.float)
        
        out = model(x_data, data.edge_attr, data.edge_index, batch=data.batch)  # Perform a single forward pass.
        loss = criterion(out, y_data)  # Compute the loss.
        r2_error += r2(out.flatten(), y_data.flatten())
        error += loss.item()
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        
        print(f'Epoch n°{epoch_id} progress: {(b_i + 1) / len(train_loader) * 100.:.2f}%' \
            f' (Loss: {error / (b_i + 1):.5f}, R²: {r2_error / (b_i + 1):.5f})', end='\r')

def test(loader):
    model.eval()

    error = 0
    r2_error = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        
        # normalize data
        x_data = embedded_data(torch.tensor(x_scaler.transform(data.x), dtype=torch.float))
        y_data = torch.tensor(y_scaler.transform(data.y), dtype=torch.float)
        
        out = model(x_data, data.edge_attr, data.edge_index, batch=data.batch)
        loss = criterion(out, y_data)
        error += loss.item()  
        r2_error += r2(out.flatten(), y_data.flatten())
    return error / len(loader), r2_error / len(loader)  # Derive ratio of correct predictions.


for epoch in range(1, 5):
    train(epoch)
    train_loss, train_r2 = test(train_loader)
    test_loss, test_r2 = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train (Loss: {train_loss:.5f}, R²: {train_r2:.5f}), '\
        f'Test (Loss: {test_loss:.5f}, R²: {test_r2:.5f})', end='\n')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (432x18 and 36x256)

In [ ]:
model_folder = 'data/models/model1'
os.makedirs('data/models/model1', exist_ok=True)
torch.save(model.state_dict(), f'{model_folder}/model.pt')
torch.save(optimizer.state_dict(), f'{model_folder}/optimizer.pt')

### Predict

In [ ]:
scene_file = 'scenes/cornell-box/scene.xml'
output_folder = 'data/model1'

scene = mi.load_file(scene_file)

# get the reference image
ref_integrator = mi.load_dict({'type': 'path', 'max_depth': max_depth})
ref_image = mi.render(scene, integrator=ref_integrator, spp = 1000)

# prepare path gnn data
gnn_integrator = mi.load_dict({'type': 'pathgnn', 'max_depth': max_depth})
params = mi.traverse(scene)
gnn_log_filename = f'{output_folder}/gnn_file_predict.path'
params['logfile'] = gnn_log_filename
params.update();

mi.render(scene, integrator=gnn_integrator, spp = 10)
container = SimpleLightGraphContainer.fromfile(gnn_log_filename, scene_file, verbose=True)
container.build_connections(n_graphs=10, n_nodes_per_graphs=5, n_neighbors=5, verbose=True)

In [ ]:
from torch_geometric.data import Data

for key in container.keys():
    graphs = container.graphs_at(key)
    for graph in graphs:
        torch_data = graph.data.to_torch()
        
        data = Data(x = torch_data.x, 
                edge_index = torch_data.edge_index,
                y = torch_data.y,
                edge_attr = torch_data.edge_attr,
                pos = torch_data.pos)
        
        data.x = (data.x - x_mean) / x_std
        data.y = (data.y - y_mean) / y_std
        
        # now predict and get value